In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gunzip /content/gdrive/My\ Drive/overall-state.csv.gz
!gunzip /content/gdrive/My\ Drive/overall-county.csv.gz

gzip: /content/gdrive/My Drive/overall-state.csv.gz: No such file or directory
gzip: /content/gdrive/My Drive/overall-county.csv.gz: No such file or directory


In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import pandas as pd
#from pandas_profiling import ProfileReport
import glob
from datetime import datetime
import time
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_squared_error,median_absolute_error,mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier
import matplotlib.pyplot as plt

import numpy as np
from itertools import combinations 
%matplotlib inline

In [ ]:
state_data = pd.DataFrame()

# Insert relevant path here
state_csv = 'drive/My Drive/overall-state.csv'
#state_csv = '/content/gdrive/My Drive/overall-state.csv'
state_data = pd.read_csv(state_csv, low_memory=False)

In [ ]:
state_data=state_data.drop(['date'],axis=1)
state_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83404 entries, 0 to 83403
Columns: 103 entries, state_code to pct_multiple_medical_conditions_weighted
dtypes: float64(99), int64(1), object(3)
memory usage: 65.5+ MB


In [ ]:
state_data.isna().sum()

state_code                                  0
gender                                      0
age_bucket                                  0
n                                           0
weight_sums                                 0
                                           ..
pct_chronic_lung_disease_weighted           0
pct_kidney_disease_weighted                 0
pct_autoimmune_disorder_weighted            0
pct_no_above_medical_conditions_weighted    0
pct_multiple_medical_conditions_weighted    0
Length: 103, dtype: int64

In [ ]:
state_data_dropped = state_data.dropna()#na values dropped
state_data_dropped.head()

,state_code,gender,age_bucket,n,weight_sums,pct_cli,pct_ili,pct_cli_anosmia_ageusia,pct_hh_cli,pct_cmnty_cli,pct_hh_fever,pct_hh_sore_throat,pct_hh_cough,pct_hh_shortness_of_breath,pct_hh_difficulty_breathing,mean_hh_cli_ct,mean_cmnty_cli_ct,pct_self_fever,pct_self_cough,pct_self_shortness_of_breath,pct_self_difficulty_breathing,pct_self_tiredness_or_exhaustion,pct_self_nasal_congestion,pct_self_runny_nose,pct_self_muscle_joint_aches,pct_self_sore_throat,pct_self_persistent_pain_pressure_in_chest,pct_self_nausea_vomiting,pct_self_diarrhea,pct_self_anosmia_ageusia,pct_self_other,pct_self_none_of_above,pct_self_multiple_symptoms,pct_tested_and_positive,pct_tested_and_negative,pct_tested_no_result,pct_could_not_get_tested,pct_did_not_try_to_get_tested,pct_worked_outside_home,pct_avoid_contact_all_or_most_time,...,pct_hh_difficulty_breathing_weighted,mean_hh_cli_ct_weighted,mean_cmnty_cli_ct_weighted,pct_self_fever_weighted,pct_self_cough_weighted,pct_self_shortness_of_breath_weighted,pct_self_difficulty_breathing_weighted,pct_self_tiredness_or_exhaustion_weighted,pct_self_nasal_congestion_weighted,pct_self_runny_nose_weighted,pct_self_muscle_joint_aches_weighted,pct_self_sore_throat_weighted,pct_self_persistent_pain_pressure_in_chest_weighted,pct_self_nausea_vomiting_weighted,pct_self_diarrhea_weighted,pct_self_anosmia_ageusia_weighted,pct_self_other_weighted,pct_self_none_of_above_weighted,pct_self_multiple_symptoms_weighted,pct_tested_and_positive_weighted,pct_tested_and_negative_weighted,pct_tested_no_result_weighted,pct_could_not_get_tested_weighted,pct_did_not_try_to_get_tested_weighted,pct_worked_outside_home_weighted,pct_avoid_contact_all_or_most_time_weighted,mean_outside_hh_contact_at_work_ct_weighted,mean_outside_hh_contact_shopping_ct_weighted,mean_outside_hh_contact_in_social_gatherings_ct_weighted,pct_contact_covid_positive_weighted,pct_diabetes_weighted,pct_cancer_weighted,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted
2350,al,female,18-34,145,462802.973425,0.0000,0.0000,2.0979,0.6897,30.4348,0.6897,13.7931,28.9655,7.5862,6.2069,0.2246,26.1159,0.0000,20.2797,6.9930,2.7972,21.6783,23.7762,18.8811,12.5874,11.8881,2.7972,5.5944,11.8881,2.0979,2.7972,48.2517,37.2414,1.3514,1.3514,1.3514,4.0541,91.8919,40.0000,75.1724,...,5.9893,0.2042,19.6547,0.0000,20.8051,6.2287,2.1679,20.9275,20.5140,18.6713,11.7630,11.8112,2.0030,4.5670,11.4994,1.3776,4.0935,47.7043,36.8184,0.5417,0.5586,0.8302,2.0494,46.6206,41.3065,75.5190,5.5642,6.4684,0.7446,1.8876,3.9268,0.3763,2.2387,9.1461,14.0375,0.0000,0.0000,3.1909,70.0717,2.2785
2351,al,female,35-54,194,583249.802288,2.0942,2.0942,4.1885,3.1579,22.7513,4.7368,9.3750,18.5567,6.2827,3.6842,190.5397,1.5661,3.1414,15.1832,5.7592,3.1414,17.8010,19.3717,14.6597,13.6126,7.3298,2.0942,6.2827,8.9005,3.1414,2.6178,61.2565,29.3814,0.0000,2.7027,1.3514,10.8108,85.1351,46.9072,71.1340,...,3.6074,243.1850,1.4147,3.3699,14.6460,5.4355,3.2747,16.2160,17.8710,13.6654,14.0172,7.0918,2.2939,5.9966,8.5825,3.0300,1.9825,62.1165,27.8368,0.0000,1.3754,0.4220,4.1728,30.6084,47.1693,70.7031,8.7415,3.9712,0.4869,2.4750,10.2654,1.4028,3.8820,30.0390,14.9271,5.7298,0.0000,10.6853,49.2702,17.9230
2352,al,female,55+,163,645561.896600,1.2500,1.2500,3.1250,0.6173,13.2075,1.2346,4.9689,19.6319,7.4534,3.1056,0.1242,0.4277,1.2500,15.0000,7.5000,3.1250,15.0000,16.2500,18.1250,16.2500,5.0000,1.8750,3.7500,9.3750,3.1250,4.3750,61.2500,28.8344,1.6393,1.6393,0.0000,3.2787,93.4426,19.6319,85.2761,...,2.6305,0.1186,0.3646,1.6240,13.8949,7.0713,2.9406,14.8100,16.0242,17.7509,16.6295,5.5636,1.8443,3.9075,8.7967,3.7496,4.3249,59.4180,28.3466,0.7340,0.4965,0.0000,1.0683,35.3323,19.2710,84.8110,1.6625,2.9751,0.1173,1.8928,21.7870,10.0665,8.4887,59.0565,15.1619,10.5929,4.0285,12.0755,21.7587,40.0636
2353,al,male,35-54,72,413211.0

In [ ]:
print (max(state_data_dropped['pct_tested_and_positive']))
print (min(state_data_dropped['pct_tested_and_positive']))
print (state_data_dropped.shape)# we are not lossing much , we are loosing around 10% of the data

50.0
0.0
(76790, 103)


In [ ]:
state_data_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76790 entries, 2350 to 83398
Columns: 103 entries, state_code to pct_multiple_medical_conditions_weighted
dtypes: float64(99), int64(1), object(3)
memory usage: 60.9+ MB


In [ ]:
print(state_data_dropped['gender'].value_counts())
print(state_data_dropped['age_bucket'].value_counts())

overall    28124
female     26466
male       22200
Name: gender, dtype: int64
overall    21875
55+        19187
35-54      19067
18-34      16661
Name: age_bucket, dtype: int64


In [ ]:
male_df=state_data_dropped[state_data_dropped['gender']=='male']
female_df=state_data_dropped[state_data_dropped['gender']=='female']
young_df=state_data_dropped[state_data_dropped['age_bucket']=='18-34']
mid_df=state_data_dropped[state_data_dropped['age_bucket']=='35-54']
old_df=state_data_dropped[state_data_dropped['age_bucket']=='55+']

In [ ]:
#normalisation
scaler = MinMaxScaler()
state_data_droppedCat=state_data_dropped.drop(['state_code','gender','age_bucket'],axis=1) #with categorical variables
#state_data_droppedCat.shape
state_data_dropped_scaled = scaler.fit_transform(state_data_droppedCat)
state_data_dropped_scaled = pd.DataFrame(state_data_dropped_scaled, columns=state_data_droppedCat.columns)
# # state_data_dropped_scaled.merge(state_data_dropped)
state_data_dropped_scaled.head()
#state_data_dropped_scaled=pd.concat([state_data_dropped_scaled,mergedf1], axis=1)
# state_data_dropped_scaled.head()
state_data_dropped_scaled.head()

,n,weight_sums,pct_cli,pct_ili,pct_cli_anosmia_ageusia,pct_hh_cli,pct_cmnty_cli,pct_hh_fever,pct_hh_sore_throat,pct_hh_cough,pct_hh_shortness_of_breath,pct_hh_difficulty_breathing,mean_hh_cli_ct,mean_cmnty_cli_ct,pct_self_fever,pct_self_cough,pct_self_shortness_of_breath,pct_self_difficulty_breathing,pct_self_tiredness_or_exhaustion,pct_self_nasal_congestion,pct_self_runny_nose,pct_self_muscle_joint_aches,pct_self_sore_throat,pct_self_persistent_pain_pressure_in_chest,pct_self_nausea_vomiting,pct_self_diarrhea,pct_self_anosmia_ageusia,pct_self_other,pct_self_none_of_above,pct_self_multiple_symptoms,pct_tested_and_positive,pct_tested_and_negative,pct_tested_no_result,pct_could_not_get_tested,pct_did_not_try_to_get_tested,pct_worked_outside_home,pct_avoid_contact_all_or_most_time,mean_outside_hh_contact_at_work_ct,mean_outside_hh_contact_shopping_ct,mean_outside_hh_contact_in_social_gatherings_ct,...,pct_hh_difficulty_breathing_weighted,mean_hh_cli_ct_weighted,mean_cmnty_cli_ct_weighted,pct_self_fever_weighted,pct_self_cough_weighted,pct_self_shortness_of_breath_weighted,pct_self_difficulty_breathing_weighted,pct_self_tiredness_or_exhaustion_weighted,pct_self_nasal_congestion_weighted,pct_self_runny_nose_weighted,pct_self_muscle_joint_aches_weighted,pct_self_sore_throat_weighted,pct_self_persistent_pain_pressure_in_chest_weighted,pct_self_nausea_vomiting_weighted,pct_self_diarrhea_weighted,pct_self_anosmia_ageusia_weighted,pct_self_other_weighted,pct_self_none_of_above_weighted,pct_self_multiple_symptoms_weighted,pct_tested_and_positive_weighted,pct_tested_and_negative_weighted,pct_tested_no_result_weighted,pct_could_not_get_tested_weighted,pct_did_not_try_to_get_tested_weighted,pct_worked_outside_home_weighted,pct_avoid_contact_all_or_most_time_weighted,mean_outside_hh_contact_at_work_ct_weighted,mean_outside_hh_contact_shopping_ct_weighted,mean_outside_hh_contact_in_social_gatherings_ct_weighted,pct_contact_covid_positive_weighted,pct_diabetes_weighted,pct_cancer_weighted,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted
0,0.013053,0.016199,0.000000,0.000000,0.175461,0.079315,0.534914,0.070350,0.482759,0.673251,0.297612,0.366206,2.672740e-36,1.741931e-137,0.000000,0.638435,0.419579,0.231767,0.489769,0.570628,0.528671,0.377622,0.457235,0.223776,0.329449,0.534965,0.186480,0.183372,0.334972,0.653113,0.027028,0.013514,0.040542,0.121623,0.918919,0.362500,0.716810,1.070113e-67,1.105042e-67,1.025677e-103,...,0.264216,1.565465e-36,1.648134e-137,0.000000,0.593999,0.256420,0.101938,0.431753,0.482105,0.494516,0.344921,0.430585,0.120891,0.197044,0.406827,0.064812,0.252952,0.337982,0.635307,0.054208,0.025295,0.070248,0.080942,0.655019,0.439324,0.770966,1.588014e-67,1.789305e-67,9.909855e-104,0.073080,0.084631,0.013062,0.059806,0.115071,0.315907,0.000000,0.000000,0.104553,0.750925,0.034961
1,0.019786,0.020952,0.308147,0.282717,0.350312,0.363157,0.399871,0.483155,0.328125,0.431317,0.246475,0.217367,2.267422e-33,1.044589e-138,0.376970,0.477990,0.345551,0.260287,0.402171,0.464920,0.410471,0.408378,0.281915,0.167536,0.369982,0.400523,0.279236,0.171612,0.540762,0.507764,0.000000,0.027027,0.040542,0.324324,0.851351,0.435889,0.670747,2.178275e-67,9.745476e-68,9.346157e-104,...,0.159139,1.864337e-33,1.186289e-138,0.264418,0.418152,0.223766,0.153981,0.334551,0.419991,0.361933,0.411019,0.258536,0.138448,0.258724,0.303632,0.142552,0.122506,0.565547,0.475305,0.000000,0.062282,0.035708,0.164806,0.430048,0.501777,0.721759,2.504295e-67,1.069154e-67,6.480135e-104,0.095821,0.221240,0.048694,0.103706,0.377935,0.335927,0.207365,0.000000,0.350115,0.504410,0.275009
2,0.015526,0.023412,0.183929,0.168750,0.261364,0.070989,0.232132,0.125929,0.173912,0.456309,0.292403,0.183230,1.477980e-36,2.852759e-139,0.150001,0.472222,0.449999,0.258928,0.338889,0.390000,0.507

In [ ]:
y= state_data_dropped_scaled['pct_tested_and_positive']
output_like_variables=['pct_tested_and_positive','pct_tested_and_negative', 'pct_tested_no_result', 'pct_could_not_get_tested', 'pct_did_not_try_to_get_tested',\
                       'pct_tested_and_positive_weighted','pct_contact_covid_positive', 'pct_contact_covid_positive_weighted' ,\
                       'pct_tested_and_negative_weighted',	'pct_tested_no_result_weighted',	'pct_could_not_get_tested_weighted',	'pct_did_not_try_to_get_tested_weighted'] 
                        
X=state_data_dropped_scaled.drop(output_like_variables,axis=1)
X.columns

Index(['n', 'weight_sums', 'pct_cli', 'pct_ili', 'pct_cli_anosmia_ageusia',
       'pct_hh_cli', 'pct_cmnty_cli', 'pct_hh_fever', 'pct_hh_sore_throat',
       'pct_hh_cough', 'pct_hh_shortness_of_breath',
       'pct_hh_difficulty_breathing', 'mean_hh_cli_ct', 'mean_cmnty_cli_ct',
       'pct_self_fever', 'pct_self_cough', 'pct_self_shortness_of_breath',
       'pct_self_difficulty_breathing', 'pct_self_tiredness_or_exhaustion',
       'pct_self_nasal_congestion', 'pct_self_runny_nose',
       'pct_self_muscle_joint_aches', 'pct_self_sore_throat',
       'pct_self_persistent_pain_pressure_in_chest',
       'pct_self_nausea_vomiting', 'pct_self_diarrhea',
       'pct_self_anosmia_ageusia', 'pct_self_other', 'pct_self_none_of_above',
       'pct_self_multiple_symptoms', 'pct_worked_outside_home',
       'pct_avoid_contact_all_or_most_time',
       'mean_outside_hh_contact_at_work_ct',
       'mean_outside_hh_contact_shopping_ct',
       'mean_outside_hh_contact_in_social_gatherings_ct', 

In [ ]:
unweighted=['n','weight_sums','pct_cli','pct_ili', 'pct_cli_anosmia_ageusia',\
        'pct_hh_cli', 'pct_cmnty_cli', 'pct_hh_fever', 'pct_hh_sore_throat',\
         'pct_hh_cough', 'pct_hh_shortness_of_breath','pct_hh_difficulty_breathing', 'mean_hh_cli_ct', 'mean_cmnty_cli_ct',\
         'pct_self_fever', 'pct_self_cough', 'pct_self_shortness_of_breath',\
         'pct_self_difficulty_breathing', 'pct_self_tiredness_or_exhaustion',\
         'pct_self_nasal_congestion', 'pct_self_runny_nose',\
         'pct_self_muscle_joint_aches', 'pct_self_sore_throat',
       'pct_self_persistent_pain_pressure_in_chest',\
       'pct_self_nausea_vomiting', 'pct_self_diarrhea',\
       'pct_self_anosmia_ageusia', 'pct_self_other', 'pct_self_none_of_above',\
       'pct_self_multiple_symptoms', 'pct_worked_outside_home',\
       'pct_avoid_contact_all_or_most_time',\
       'mean_outside_hh_contact_at_work_ct',\
       'mean_outside_hh_contact_shopping_ct',\
       'mean_outside_hh_contact_in_social_gatherings_ct', 'pct_diabetes',\
       'pct_cancer', 'pct_heart_disease', 'pct_high_blood_pressure', \
       'pct_asthma', 'pct_chronic_lung_disease', 'pct_kidney_disease',\
       'pct_autoimmune_disorder', 'pct_no_above_medical_conditions',\
       'pct_multiple_medical_conditions', ]
X=X.drop(unweighted,axis=1)
n_features = X.shape[1]
print(n_features)

43


In [ ]:
derived=['pct_ili_weighted','pct_cli_weighted','pct_cli_anosmia_ageusia_weighted',]
X=X.drop(derived,axis=1)
n_features = X.shape[1]
print(n_features)

40


In [ ]:
X.columns

Index(['pct_hh_cli_weighted', 'pct_cmnty_cli_weighted',
       'pct_hh_fever_weighted', 'pct_hh_sore_throat_weighted',
       'pct_hh_cough_weighted', 'pct_hh_shortness_of_breath_weighted',
       'pct_hh_difficulty_breathing_weighted', 'mean_hh_cli_ct_weighted',
       'mean_cmnty_cli_ct_weighted', 'pct_self_fever_weighted',
       'pct_self_cough_weighted', 'pct_self_shortness_of_breath_weighted',
       'pct_self_difficulty_breathing_weighted',
       'pct_self_tiredness_or_exhaustion_weighted',
       'pct_self_nasal_congestion_weighted', 'pct_self_runny_nose_weighted',
       'pct_self_muscle_joint_aches_weighted', 'pct_self_sore_throat_weighted',
       'pct_self_persistent_pain_pressure_in_chest_weighted',
       'pct_self_nausea_vomiting_weighted', 'pct_self_diarrhea_weighted',
       'pct_self_anosmia_ageusia_weighted', 'pct_self_other_weighted',
       'pct_self_none_of_above_weighted',
       'pct_self_multiple_symptoms_weighted',
       'pct_worked_outside_home_weighted',
 

In [ ]:
X.head()

,pct_hh_cli_weighted,pct_cmnty_cli_weighted,pct_hh_fever_weighted,pct_hh_sore_throat_weighted,pct_hh_cough_weighted,pct_hh_shortness_of_breath_weighted,pct_hh_difficulty_breathing_weighted,mean_hh_cli_ct_weighted,mean_cmnty_cli_ct_weighted,pct_self_fever_weighted,pct_self_cough_weighted,pct_self_shortness_of_breath_weighted,pct_self_difficulty_breathing_weighted,pct_self_tiredness_or_exhaustion_weighted,pct_self_nasal_congestion_weighted,pct_self_runny_nose_weighted,pct_self_muscle_joint_aches_weighted,pct_self_sore_throat_weighted,pct_self_persistent_pain_pressure_in_chest_weighted,pct_self_nausea_vomiting_weighted,pct_self_diarrhea_weighted,pct_self_anosmia_ageusia_weighted,pct_self_other_weighted,pct_self_none_of_above_weighted,pct_self_multiple_symptoms_weighted,pct_worked_outside_home_weighted,pct_avoid_contact_all_or_most_time_weighted,mean_outside_hh_contact_at_work_ct_weighted,mean_outside_hh_contact_shopping_ct_weighted,mean_outside_hh_contact_in_social_gatherings_ct_weighted,pct_diabetes_weighted,pct_cancer_weighted,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted
0,0.036217,0.524708,0.036217,0.438371,0.660763,0.234115,0.264216,1.565465e-36,1.648134e-137,0.000000,0.593999,0.256420,0.101938,0.431753,0.482105,0.494516,0.344921,0.430585,0.120891,0.197044,0.406827,0.064812,0.252952,0.337982,0.635307,0.439324,0.770966,1.588014e-67,1.789305e-67,9.909855e-104,0.084631,0.013062,0.059806,0.115071,0.315907,0.000000,0.000000,0.104553,0.750925,0.034961
1,0.222550,0.419016,0.341577,0.303130,0.380600,0.202959,0.159139,1.864337e-33,1.186289e-138,0.264418,0.418152,0.223766,0.153981,0.334551,0.419991,0.361933,0.411019,0.258536,0.138448,0.258724,0.303632,0.142552,0.122506,0.565547,0.475305,0.501777,0.721759,2.504295e-67,1.069154e-67,6.480135e-104,0.221240,0.048694,0.103706,0.377935,0.335927,0.207365,0.000000,0.350115,0.504410,0.275009
2,0.049074,0.215560,0.085411,0.172357,0.404429,0.239765,0.116044,9.092271e-37,3.057333e-139,0.127427,0.396708,0.291108,0.138271,0.305544,0.376589,0.470139,0.487619,0.202825,0.111312,0.168590,0.311210,0.176407,0.267251,0.522939,0.484387,0.204593,0.865909,4.628270e-68,7.818947e-68,1.561142e-104,0.469554,0.349428,0.226771,0.743018,0.341211,0.383363,0.175611,0.395666,0.178376,0.614732
3,0.000000,0.284140,0.000000,0.408686,0.445135,0.345651,0.204485,1.112385e-36,6.602698e-139,0.000000,0.487302,0.346754,0.217958,0.276054,0.691677,0.570336,0.537185,0.343481,0.425918,0.000000,0.389810,0.000000,0.226233,0.330494,0.699285,0.581923,0.717683,1.458356e-67,1.835014e-67,1.054336e-103,0.263045,0.051613,0.036775,0.551448,0.242642,0.212598,0.000000,0.190413,0.444863,0.310264
4,0.206651,0.145362,0.206651,0.067312,0.353338,0.206496,0.118677,3.451163e-33,1.748365e-139,0.160131,0.239657,0.139743,0.126497,0.145722,0.245924,0.332916,0.207691,0.054873,0.044807,0.128961,0.137759,0.000000,0.347540,0.718279,0.316710,0.260088,0.854851,1.414781e-67,8.707745e-68,0.000000e+00,0.508301,0.401343,0.622009,0.739097,0.158976,0.330568,0.104739,0.212036,0.231736,0.683474


In [ ]:
mean_D = ['mean_outside_hh_contact_in_social_gatherings_ct_weighted','mean_cmnty_cli_ct_weighted',\
        'mean_outside_hh_contact_shopping_ct_weighted','mean_outside_hh_contact_at_work_ct_weighted']
X=X.drop(mean_D,axis=1)
n_features = X.shape[1]
print(n_features)

36


In [ ]:
y_new=state_data_dropped['pct_tested_and_positive']
X_new = state_data_dropped.drop(output_like_variables,axis=1)
X_new = X_new.drop(unweighted,axis=1)
X_new=X_new.drop(derived,axis=1)
X_new=X_new.drop(mean_D,axis=1)
y_new.describe()

count    76790.000000
mean         1.530617
std          2.030489
min          0.000000
25%          0.000000
50%          0.865800
75%          2.319600
max         50.000000
Name: pct_tested_and_positive, dtype: float64

In [ ]:
selected_10 =['pct_hh_cli_weighted', 'pct_hh_fever_weighted', 'pct_self_fever_weighted', 'pct_avoid_contact_all_or_most_time_weighted', 'pct_self_runny_nose_weighted', 'pct_worked_outside_home_weighted', 'pct_self_none_of_above_weighted', 'pct_cmnty_cli_weighted', 'pct_self_anosmia_ageusia_weighted', 'pct_hh_difficulty_breathing_weighted']  
X_train, X_test, y_train, y_test = train_test_split(X_new[selected_10], y_new, test_size=0.2, random_state=42)
reg = GradientBoostingRegressor()
reg = reg.fit(X_train,y_train)
y_pred= reg.predict(X_test)
yp_l=list(y_pred)
yt_l=list(y_test)
pct_sm=0
abs_sum = 0
for j in range(len(y_pred)):
    diff=abs(yp_l[j]-yt_l[j])
    abs_sum += diff
    pct= diff/(yt_l[j]+1)
    pct_sm+= pct
rel_error=pct_sm/len(y_pred)
abs_error = abs_sum/len(y_pred)
print(abs_error,rel_error*100)

1.1440706460363106 60.84447626038462


In [ ]:
selected_10 =['pct_hh_fever_weighted', 'pct_self_fever_weighted', 'pct_avoid_contact_all_or_most_time_weighted', 'pct_self_runny_nose_weighted', 'pct_worked_outside_home_weighted', 'pct_self_none_of_above_weighted', 'pct_cmnty_cli_weighted', 'pct_self_anosmia_ageusia_weighted', 'pct_hh_difficulty_breathing_weighted']  
X_train, X_test, y_train, y_test = train_test_split(X_new[selected_10], y_new, test_size=0.2, random_state=42)
reg = GradientBoostingRegressor()
reg = reg.fit(X_train,y_train)
y_pred= reg.predict(X_test)
yp_l=list(y_pred)
yt_l=list(y_test)
pct_sm=0
abs_sum = 0
for j in range(len(y_pred)):
    diff=abs(yp_l[j]-yt_l[j])
    abs_sum += diff
    pct= diff/(yt_l[j]+1)
    pct_sm+= pct
rel_error=pct_sm/len(y_pred)
abs_error = abs_sum/len(y_pred)
print(abs_error,rel_error*100)

1.1455356685626048 60.936718664110956


In [ ]:
selected_10 =['pct_hh_cli_weighted', 'pct_hh_fever_weighted', 'pct_self_fever_weighted', 'pct_avoid_contact_all_or_most_time_weighted', 'pct_self_runny_nose_weighted', 'pct_worked_outside_home_weighted', 'pct_self_none_of_above_weighted', 'pct_cmnty_cli_weighted', 'pct_self_anosmia_ageusia_weighted', 'pct_hh_difficulty_breathing_weighted']  
X_train, X_test, y_train, y_test = train_test_split(X_new[selected_10], y_new, test_size=0.2, random_state=42)
reg = GradientBoostingRegressor()
reg = reg.fit(X_train,y_train)
y_pred= reg.predict(X_test)
yp_l=list(y_pred)
yt_l=list(y_test)
pct_sm=0
abs_sum = 0
for j in range(len(y_pred)):
    diff=abs(yp_l[j]-yt_l[j])
    abs_sum += diff
    pct= diff/(yt_l[j]+1)
    pct_sm+= pct
rel_error=pct_sm/len(y_pred)
abs_error = abs_sum/len(y_pred)
print(abs_error,rel_error*100)

1.1440853652001148 60.84323875995364
